In [1]:
# BASIC IMPORTS OF ALL LIBRARIES
'''
Glove Imports are 300-dim from GENSIM
Pickle saves all the datasets
'''
import pygraphviz
from nltk import pos_tag
import csv
import re
import numpy as np
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from collections import defaultdict
import networkx as nx
from copy import deepcopy
import pickle
from entity_groups import *
from graphviz import Source
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist 
import gensim.downloader as api
from networkx.algorithms.community import greedy_modularity_communities
from sklearn.preprocessing import normalize
from datetime import datetime, timedelta

wnl = WordNetLemmatizer()
word_vectors = api.load("glove-wiki-gigaword-300")

In [92]:
# junk verbs that are generalizably illegal
junk_rel = ["be","have","do","let","","make","tell","say","describe","decide","want","name","know","think","try","become","oneday","put","come",'see', 'need', 'look', 'help', 'come', 'take', 'get', 'put', 'pick', 'turn', 'go', 'stand', 'give', 'notice', 'use',"get","start"]

# hypothetical words that are generally useless / qualifiers
junk_words = ["would","could","should","maybe","perhaps","think","might","assume","claim","this"]

# negative sentences
junk_words.extend(['n\'t',"not"])

# perspective actants that are pointless
junk = ["you","i","we","the","it","he","she","steinbeck","people","author",
        "book","me","steinback","him","her","their","this","","shelley","mary","harper","lee","tolkien","bjp","delhi","man","men","giannulli","loughlin","singh","ryan"]

In [93]:
DATE = "0417"
file_name = "GDELT/"
text = "GDELT"

In [94]:
import pandas as pd
entity_file = "NER_list.csv"
S_use = {}
# fill this in

places = []
people = []
orgs = []

ent_file = pd.read_csv(entity_file)
for row in ent_file.head(n=100).iterrows():
    S_use[row[1]['entity']] = row[1]['entity']

for row in ent_file.head(n=250).iterrows():
    if row[1]['type'] == "PERSON":
        people.append(row[1]['entity'])
        S_use[row[1]['entity']] = row[1]['entity']

for row in ent_file.head(n=250).iterrows():
    if row[1]['type'] == "GPE":
        places.append(row[1]['entity'])
        S_use[row[1]['entity']] = row[1]['entity']

for row in ent_file.head(n=250).iterrows():
    if row[1]['type'] == "ORG":
        orgs.append(row[1]['entity'])
        S_use[row[1]['entity']] = row[1]['entity']

ads = ["youtube","twitter","facebook","reddit","4chan","8chan","instagram"]
hardcode = ["5g","radio","waves","jews","army","nanowires","alien","aliens","towers","bleach","garlic"]

for a in ads:
    S_use[a] = a

for a in hardcode:
    S_use[a] = a
    
S_use["alex"] = "alex jones"
S_use["jones"] = "alex jones"
S_use["virus"] = "covid19"
S_use["coronavirus"] = "covid19"
S_use["covid19"] = "covid19"
S_use["theory"] = "conspiracy theory"
S_use["theories"] = "conspiracy theory"
S_use["conspiracy"] = "conspiracy theory"
S_use["trumps"] = S_use["trump"]

In [95]:
path = "ExtractedGDELTData/"
date_start = "20200101000001"
date_end = "20200410000001"
#date_end = "20200101000001"
date = date_start
results = "Results0421"

In [96]:
def extract_traj(file_name_re):
    trajectories = []
    same_post = 0
    traj = ["START"]

    with open(file_name_re) as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')

        for row in readCSV:
            FLAG = 0
            sentence_num = row[1]
            sub_r = ""
            obj_r = ""
            sub_verb = ""

            # if hypothetical_sentence(row[2]):
            #     continue

            try:
                sub = re.search(r"\{(\w+)\}", row[3]).group(1).lower()
                rel = wnl.lemmatize(re.search(r"\{(\w+)\}", row[4]).group(1).lower(), pos='v')
                obj = re.search(r"\{(\w+)\}", row[5]).group(1).lower()
            except:
                continue

            words = row[2].split()
            rel_words = []

            if rel == None or obj == None or sub == None:
                continue
            
            if same_post == row[0]:
                pass
            else:
                same_post = row[0]
                traj.append("TERMINATE")
                trajectories.append(traj)
                traj = ["START"]

            try:
                sub_r = S_use[sub]
            except:
                sub_r = sub
                FLAG += 1
            
            try:
                obj_r = S_use[obj]
            except:
                obj_r = obj
                FLAG += 1

            sub_verb = ""

            try:
                sub_verb = C_use[rel]
            except:
                sub_verb = rel

            if sub_r == obj_r or sub_r in junk or obj_r in junk or sub_verb in junk_rel \
                                    or FLAG > 0 or sub_r.isdigit() or obj_r.isdigit():
                continue

            traj.append(sub_verb)
            traj.append(sub_r + "_" + obj_r + "_" + sentence_num)

    # if we missed out on last post
    if traj != "START":
        traj.append("TERMINATE")
        trajectories.append(traj)
    return trajectories

In [97]:
def find_mat(trajectories):
    rev_dict = {}
    precedence_matrix = np.zeros((50000,50000))
    dictionary_of_labels = {"START":0, "TERMINATE":1}
    counter_of_labels = defaultdict(int)
    max_rows = 1
    max_cols = 1
    counter = 2

    for trajectory in trajectories:

        triplets = ["START"]
        index = 1

        while index < len(trajectory)-2:
            triplets.append(trajectory[index] + '_' + trajectory[index + 1])
            index += 2

        triplets.append("TERMINATE")

        for key,_ in enumerate(triplets[1:-1]):
            removed_triplets = triplets[key+1].split("_")
            if len(removed_triplets) > 1:
                subject_ = removed_triplets[1]
                object_ = removed_triplets[2]
                if subject_ not in dictionary_of_labels:
                    dictionary_of_labels[subject_] = counter
                    counter += 1
                if object_ not in dictionary_of_labels:
                    dictionary_of_labels[object_] = counter
                    counter += 1
        
        triplet_temp = []

        # find uniques
        for trip in triplets:
            try:
                removed_triplet = trip.rsplit("_",1)[0]
            except:
                removed_triplet = trip

            if removed_triplet not in [t.rsplit("_",1)[0] for t in triplet_temp]:
                triplet_temp.append(trip)

        triplets = triplet_temp

        # for trip in triplets:
        #     counter_of_labels[trip] += 1

        alpha = 1.0

        for key in range(1,len(triplets)):
            removed_triplets = triplets[key].split("_")
            try:
                subject_ = removed_triplets[1]
                object_ = removed_triplets[2]
                precedence_matrix[dictionary_of_labels[subject_]]\
                                            [dictionary_of_labels[object_]] += 1.0
                precedence_matrix[dictionary_of_labels[object_]]\
                                            [dictionary_of_labels[subject_]] += 1.0
            except:
                pass


    precedence_matrix = precedence_matrix[:counter,:counter]
    for key in dictionary_of_labels:
        rev_dict[dictionary_of_labels[key]] = key

    return precedence_matrix, dictionary_of_labels, rev_dict

In [98]:
def save_partitions(G, rev_dict, date_):
    list_of_comms = greedy_modularity_communities(G)
    total_l = []
    with open(results + "/communities.txt","a+") as f:
        for l in list_of_comms:
            temp_list = []
            for i in l:
                if i not in [0,1]:
                    temp_list.append(rev_dict[i])

            if temp_list != []:
                str_l = ', '.join(temp_list)
                total_l.append("{" + str_l + "}")
        final_str = ", ".join(total_l) + "\n"
        f.write(date_ + " -> " + final_str)

In [99]:
def plot_fig(trajectories,label):    
    sub_t = trajectories
    mat, dic, rev_dict = find_mat(sub_t)

    mat = normalize(mat, 'l1')
    for i in range(0, mat.shape[0]):
        for j in range(0, mat.shape[1]):
            if mat[i][j] > mat[j][i]:
                mat[j][i] = 0

            elif mat[i][j] < mat[j][i]:
                mat[i][j] = 0

            elif mat[i][j] == mat[j][i] and i != j and mat[j][i] != 0:
                mat[j][i] = 0

    G = nx.from_numpy_matrix(mat, create_using=nx.Graph)

    save_partitions(G, rev_dict, label[4:8])

    DISP_THRESH = 2

    bb = {}
    for index, val in np.ndenumerate(mat):
        if val > 1:
            bb[index] = {"color":"red", "penwidth":2}
        else:
            bb[index] = {"color":"blue", "penwidth":0.5}
    nx.set_edge_attributes(G, bb)

    bb1 = {}
    for j in range(mat.shape[0]):
        if rev_dict[j] in places:
            bb1[j] = {"color":"green", "penwidth":4}
        if rev_dict[j] in people:
            bb1[j] = {"color":"yellow", "penwidth":4}
        if rev_dict[j] in orgs:
            bb1[j] = {"color":"black", "penwidth":4}
        if rev_dict[j] in ads:
            bb1[j] = {"color":"red", "penwidth":4}
        if rev_dict[j] in hardcode:
            bb1[j] = {"color":"pink", "penwidth":4}
        if rev_dict[j] == "covid19":
            bb1[j] = {"color":"orange", "penwidth":6}

    nx.set_node_attributes(G, bb1)

    H = nx.relabel_nodes(G, rev_dict)
    H.remove_nodes_from(list(nx.isolates(H)))
    A = nx.nx_agraph.to_agraph(H)

    A.layout('dot')
    A.draw(results + "/Actant_TimeSeries/step" + str(label) + '.png')

In [100]:
def to_date(DATE):
    return datetime.strptime(DATE, '%Y%m%d%H%M%S')

def from_date(DATE):
    return DATE.strftime("%Y%m%d%H%M%S")

def create_range(DATE):
    list_dates = []
    for i in range(5):
        list_dates.append(DATE + timedelta(days = i))
    return list_dates

In [101]:
date_real = to_date(date)
date_end_real = to_date(date_end)

while date_real <= date_end_real:    
    trajs = []
    date_margin = create_range(date_real)

    for d in date_margin:
            ner_file = path + from_date(d) + "/into_relex_relations_-1.csv"
            # print(ner_file)
            trajs.extend(extract_traj(ner_file))

    plot_fig(trajs, from_date(date_real))
    date_real += timedelta(days = 1)

0416/20200202000001/into_relex_relations_-1.csv
Extracted_0416/20200203000001/into_relex_relations_-1.csv
Extracted_0416/20200204000001/into_relex_relations_-1.csv
Extracted_0416/20200205000001/into_relex_relations_-1.csv
Extracted_0416/20200206000001/into_relex_relations_-1.csv
Extracted_0416/20200203000001/into_relex_relations_-1.csv
Extracted_0416/20200204000001/into_relex_relations_-1.csv
Extracted_0416/20200205000001/into_relex_relations_-1.csv
Extracted_0416/20200206000001/into_relex_relations_-1.csv
Extracted_0416/20200207000001/into_relex_relations_-1.csv
Extracted_0416/20200204000001/into_relex_relations_-1.csv
Extracted_0416/20200205000001/into_relex_relations_-1.csv
Extracted_0416/20200206000001/into_relex_relations_-1.csv
Extracted_0416/20200207000001/into_relex_relations_-1.csv
Extracted_0416/20200208000001/into_relex_relations_-1.csv
Extracted_0416/20200205000001/into_relex_relations_-1.csv
Extracted_0416/20200206000001/into_relex_relations_-1.csv
Extracted_0416/202002070

In [ ]:
communities = "Results0420/communities.txt"
with open(communities, "r") as f:
    list_lines = f.readlines()
    for l in list_lines:
        G=nx.Graph()
        match  = re.findall(r'{(.+?)}', l)
        G.add_nodes_from(match, shape='ellipse')
        A = nx.nx_agraph.to_agraph(G)
        A.layout('circo')
        A.draw(results + "/Communities/step" + str(l[0:4]) + '.png')